In [1]:
import os

In [2]:
pwd

'/Users/vatsal/Machine Learning/MLOPS/End_to_end_project/MLOPS_project/research'

In [26]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Vatsal-Founder/MLOPS_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Vatsal-Founder"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "69bb8e040b594c9b286a371e8bffed3b11ab2423"

In [5]:
os.chdir("../")

In [6]:
pwd

'/Users/vatsal/Machine Learning/MLOPS/End_to_end_project/MLOPS_project'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [14]:
from src.MLOPS.constants import *
from src.MLOPS.utils.common import read_yaml, create_directories,save_json

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Vatsal-Founder/MLOPS_project.mlflow"


        )
        return model_evaluation_config


In [21]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlflow.models import infer_signature
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        signature=infer_signature(test_x,test_y)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model", signature=signature)


In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    print("Error occurred during model evaluation:", e)
    raise e

[2025-08-16 16:23:25,011: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-16 16:23:25,012: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-16 16:23:25,014: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-16 16:23:25,014: INFO: common: created directory at: artifacts]
[2025-08-16 16:23:25,015: INFO: common: created directory at: artifacts/model_evaluation]


/Users/vatsal/Machine Learning/MLOPS/End_to_end_project/MLOPS_project/venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


[2025-08-16 16:23:25,310: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
🏃 View run upbeat-zebra-258 at: https://dagshub.com/Vatsal-Founder/MLOPS_project.mlflow/#/experiments/0/runs/4151659930a24386a12bfce18e648afb
🧪 View experiment at: https://dagshub.com/Vatsal-Founder/MLOPS_project.mlflow/#/experiments/0
